In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
nodes_on_each_layer = [784, 15, 10]
number_of_hidden_layers = len(nodes_on_each_layer) - 2
batch_size = 32
learning_rate = 0.1
epochs = 10
train_percent = 0.9

In [ ]:
def ReLU(Z):
    return np.maximum(Z, 0)

def ReLU_derivative(Z):
    return Z > 0

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

In [ ]:
def prepare_data(data, train_percent):
    data = np.array(data)
    m, n = data.shape
    # m - number of tests
    # n - number of pixels
    sz_train = int(train_percent * m)
    sz_test = m - sz_train
    
    np.random.shuffle(data)

    data_test = data[:sz_test].T
    Y_test = data_test[0]
    X_test = data_test[1:].T / 255.
    
    data_train = data[sz_test:].T
    Y_train = data_train[0]
    X_train = data_train[1:].T / 255.
    return X_train, Y_train, X_test, Y_test

# data_train, X_train, Y_train, len(X_train)

In [ ]:
def init_params(number_of_hidden_layers, nodes_on_each_layer):
    layer_biases = []
    weights = []
    for i in range(number_of_hidden_layers + 1):
        aux = np.random.rand(nodes_on_each_layer[i + 1]) - 0.5
        layer_biases.append(aux)
    for i in range(number_of_hidden_layers + 1):
        w = np.random.rand(nodes_on_each_layer[i], nodes_on_each_layer[i + 1]) - 0.5
        weights.append(w)
    return layer_biases, weights

In [ ]:
def forward_propagation(number_of_hidden_layers, nodes_on_each_layer, weights, layer_biases, X):
    A = []
    Z = []
    prev_a = X
    for i in range(number_of_hidden_layers):
        z = np.dot(prev_a, weights[i]) + layer_biases[i]
        a = ReLU(z)
        A.append(a)
        Z.append(z)
        prev_a = a
    # apply sigmoid activation for the output layer
    z = np.dot(prev_a, weights[-1]) + layer_biases[-1]   
    a = sigmoid(z)
    A.append(a)
    Z.append(z)
#     print(type(A), A)
#     print(type(Z), Z)
#     print("----------")
    return A, Z

In [ ]:
def backward_propagation(A, Z, weights, layer_biases, X, Y):
    one_hot = np.zeros((len(Y), 10))
    for i in range(len(Y)):
        one_hot[i][Y[i]] = 1
    batch_size = len(Y)
    sz = len(A) - 1
#     print(len(A), len(A[-1]), len(A[-1][-1]))
    dW = [None] * len(A)
    dB = [None] * len(A)
#     print(f'one hot shape: {one_hot.shape}')
#     print(f'output layer shape: {A[-1].shape}')
    for layer in range(sz, -1, -1):
#         print(f'layer: {layer}')
        derivative = sigmoid_derivative(Z[layer]) if layer == sz else ReLU_derivative(Z[layer])
#         print(f'derivative shape: {derivative.shape}')
#         print(f'weights[layer] shape: {weights[layer].shape}')        
        db = derivative * np.dot(db, weights[layer + 1].T) if layer != sz else (A[layer] - one_hot) * derivative
        dB[layer] = np.mean(db, axis=0)
#         print(f'db shape: {db.shape}')
        n = A[layer - 1].shape[1] if layer != 0 else len(X[-1])
        dw = np.zeros((n, db.shape[1]))
#         print(f'dw shape: {dw.shape}')
        # can we use numpy for this instead of iterating with for?
        for k in range(batch_size):
#             print(len(A[layer - 1][k]))
            aux = np.array(A[layer - 1][k]) if layer != 0 else np.array(X[k])
#             print(f'aux shape: {aux.shape}')
#             print(f'db[k] shape: {db[k].shape}')
            dw += np.outer(aux, db[k])
        dW[layer] = dw / batch_size
    return dW, dB

In [ ]:
def update_params(weights, layer_biases, dW, dB, alpha):
#     print('update params')
#     print(len(weights), len(dW))
    for i in range(len(weights)):
        weights[i] = weights[i] - alpha * dW[i]
        layer_biases[i] = layer_biases[i] - alpha * dB[i]
    return weights, layer_biases

In [ ]:
def get_predictions(A):
    return np.argmax(A, 1)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [ ]:
def loss_function(A, Y):
    return np.mean(((Y - A) ** 2) / 2)

In [ ]:
def gradient_descent(X, Y, alpha, epochs, number_of_hidden_layers, nodes_on_each_layer, batch_size):
    layer_biases, weights = init_params(number_of_hidden_layers, nodes_on_each_layer)
    for i in range(epochs):
        for j in range(len(X)):
            x = X[j]
            y = Y[j]
            A, Z = forward_propagation(number_of_hidden_layers, nodes_on_each_layer, weights, layer_biases, x)
            dW, dB = backward_propagation(A, Z, weights, layer_biases, x, y)
            weights, layer_biases = update_params(weights, layer_biases, dW, dB, alpha)
        print(f'epoch: {i}')
        A, _ = forward_propagation(number_of_hidden_layers, nodes_on_each_layer, weights, layer_biases, x)
        predictions = get_predictions(A[-1])
        print(f'accuracy: {get_accuracy(predictions, y)}')
        print(f'loss function: {loss_function(predictions, y)}')
    return weights, layer_biases

In [ ]:
data = pd.read_csv('/kaggle/input/mnist-digit-recognizer/train.csv')
X_train, Y_train, X_test, Y_test = prepare_data(data, train_percent)

In [ ]:
# if we shuffle the data at each epoch we need to split into batches everytime :/
sz = len(X_train) / batch_size
batch_X = np.array_split(X_train, sz)
batch_Y = np.array_split(Y_train, sz) 
# number of nodes for the input layer must be 784 and for the output layer 10 for this dataset
# train data, desired output, alpha, epochs, hidden layers, nodes on layers
weights, layer_biases = gradient_descent(batch_X, batch_Y, learning_rate, epochs, number_of_hidden_layers, nodes_on_each_layer, batch_size)

In [ ]:
A, _ = forward_propagation(number_of_hidden_layers, nodes_on_each_layer, weights, layer_biases, X_test)
predictions = get_predictions(A[-1])
print(f'accuracy: {get_accuracy(predictions, Y_test)}')
print(f'loss function: {loss_function(predictions, Y_test)}')